The ICAR (Intrinsic Conditional Autoregressive) model is a statistical model commonly used in spatial statistics to analyze and model spatial data. It is specifically used to account for spatial dependencies or autocorrelation in the data, where observations in close proximity tend to be more similar than those farther apart. The ICAR model is a type of conditional autoregressive model that assumes that the value of a location in a spatial dataset depends on the values of its neighboring locations.

Here are the key components of the ICAR model:

- Neighborhood Structure: The ICAR model requires a predefined neighborhood structure. This structure specifies which locations are considered neighbors of each other. Common choices include defining neighbors based on contiguity, distance thresholds, or some other criteria.

- Conditional Autoregressive Structure: In the ICAR model, each location's value is assumed to be conditionally dependent on the values of its neighbors. This conditional dependency is often modeled as a weighted sum of the neighboring values, where the weights are determined by the neighborhood structure. The weights are typically constrained such that they sum to zero, ensuring stationarity of the model.

- Intrinsic Condition: The "intrinsic" part of the ICAR model refers to the fact that the sum of the weights for each location is constrained to be zero. This constraint helps ensure identifiability of the model parameters.

The ICAR model is useful in various spatial applications, such as spatial interpolation, spatial smoothing, and spatial prediction. It is commonly employed in Bayesian spatial modeling, where prior distributions are assigned to the model parameters, and the posterior distribution is obtained through Bayesian inference methods like Markov Chain Monte Carlo (MCMC) sampling.

By accounting for spatial dependencies through the ICAR model, statisticians and researchers can better analyze and model spatial data while considering the inherent spatial correlation that often exists in such datasets.

To implement the ICAR (Intrinsic Conditional Autoregressive) model in NumPyro, you can define a custom NumPyro model using the Pyro probabilistic programming framework, which NumPyro is built on. Here's an example of how you can write the ICAR model in NumPyro:

In [4]:
import jax
import numpy as np
import numpyro
import numpyro.distributions as dist
import jax.numpy as jnp
from numpyro.infer import MCMC, NUTS

def icar_model(y, neighbors):
    n = len(y)
    alpha = numpyro.sample('alpha', dist.Normal(0, 1))
    sigma = numpyro.sample('sigma', dist.HalfCauchy(1))
    
    # Calculate the precision matrix
    tau = 1.0 / (sigma**2)
    precision_matrix = tau * (jnp.diag(jnp.sum(neighbors, axis=1)) - neighbors)
    
    # Define the conditional autoregressive structure
    with numpyro.plate('data', n):
        mu = alpha + jnp.matmul(precision_matrix, y)
        y = numpyro.sample('y', dist.MultivariateNormal(mu, precision_matrix), obs=y)

# Example usage:
# Define your spatial data 'y' and neighbor structure 'neighbors'
y = np.array([1.2, 2.5, 3.8, 4.1, 5.2])  # Replace with your data
neighbors = np.array([[0, 1, 0, 0, 0],
                      [1, 0, 1, 0, 0],
                      [0, 1, 0, 1, 0],
                      [0, 0, 1, 0, 1],
                      [0, 0, 0, 1, 0]])  # Replace with your neighbor structure

# Run MCMC to infer the parameters
nuts_kernel = NUTS(icar_model)
mcmc = MCMC(nuts_kernel, num_samples=1000, num_warmup=1000)
mcmc.run(rng_key=jax.random.PRNGKey(0), y=y, neighbors=neighbors)

# Extract posterior samples
samples = mcmc.get_samples()


sample: 100%|██████████| 2000/2000 [00:02<00:00, 779.69it/s, 3 steps of size 7.13e-08. acc. prob=0.91] 


In [10]:
import numpyro.distributions as dist
import jax
import jax.numpy as jnp
from numpyro.infer import MCMC, NUTS

# Define the mean and covariance matrix for the multivariate normal distribution
mean = jnp.array([0.0, 0.0])  # Mean vector
cov_matrix = jnp.array([[1.0, 0.5], [0.5, 2.0]])  # Covariance matrix

# Create a MultivariateNormal distribution
mvn = dist.MultivariateNormal(mean, cov_matrix)

# Sample from the distribution
samples = mvn.sample(key=jax.random.PRNGKey(0), sample_shape=(1000,))

samples.shape

# Now, 'samples' contains 1000 samples from the multivariate normal distribution


(1000, 2)

In this code:

- We define the ICAR model as a Python function icar_model that takes y (the spatial data) and neighbors (the neighbor structure) as arguments.

- Inside the model, we sample the parameters alpha (the intercept) and sigma (the spatial correlation parameter).

- We use a plate to handle multiple data points if needed. The core of the model is the calculation of mu, which represents the conditional expectation based on the ICAR structure.

- Finally, we sample y from a Normal distribution with mean mu and observation noise of 1.

- We provide example usage of the model, where you should replace the y and neighbors arrays with your actual data and neighbor structure.

- We run MCMC using the NUTS sampler to infer the posterior distribution of the model parameters.

- After running MCMC, you can extract posterior samples from the mcmc object.

Make sure to adapt this code to your specific dataset and spatial structure as needed.